In [2]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
from tensorflow.contrib import learn
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
from utils import *

%matplotlib inline

In [3]:
sess=tf.Session()    
#First let's load meta graph and restore weights
saver = tf.train.import_meta_graph('model-9500.meta')
saver.restore(sess,tf.train.latest_checkpoint('./'))
 
graph = tf.get_default_graph()

INFO:tensorflow:Restoring parameters from /Users/adb/Google Drive/Data Analytics/project/xtalkdb/Deep Learning/num_filter_64_fold_0/1512825696/checkpoints/model-9900


In [4]:
# Load data
x_text, y = load_data_and_labels()

# Build vocabulary
max_document_length = max([len(x.split(" ")) for x in x_text])
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
x = np.array(list(vocab_processor.fit_transform(x_text)))
print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))


encodedPathwayA, encodedPathwayB = list(vocab_processor.transform(['pathwayA pathwayB']))[0][:2]
print("encodedPathwayA = %s" % encodedPathwayA, "encodedPathwayB = %s" % encodedPathwayB)

word_distancesA = load_word_distancesA()
word_distancesB = load_word_distancesB()

pos_embedding = load_pos_embedding()

Vocabulary Size: 33447
encodedPathwayA = 8 encodedPathwayB = 53


In [8]:
pathway_to_genes_dict = pickle.load(open( "data/pathway_to_genes_dict.p", "rb" ))
sentence_support_df = pd.read_csv('data/sentence_support_v3.tsv', delimiter='\t')
sentence_support_df.drop_duplicates(inplace=True)
sentence_support_df['tokenizedSentenceFromPaper'] = sentence_support_df.apply(lambda x: tokenize_pathway_names(x.sentenceFromPaper, x.pathwayA, x.pathwayB), axis=1)
sentence_support_df['predictedLabel'] = sentence_support_df.apply(lambda x: x.label, axis=1)

In [5]:
sess.graph.get_operations()

[<tf.Operation 'input_x' type=Placeholder>,
 <tf.Operation 'input_y' type=Placeholder>,
 <tf.Operation 'dropout_keep_prob' type=Placeholder>,
 <tf.Operation 'word_distancesA' type=Placeholder>,
 <tf.Operation 'word_distancesB' type=Placeholder>,
 <tf.Operation 'encoded_pos' type=Placeholder>,
 <tf.Operation 'Const' type=Const>,
 <tf.Operation 'word_embedding/random_uniform/shape' type=Const>,
 <tf.Operation 'word_embedding/random_uniform/min' type=Const>,
 <tf.Operation 'word_embedding/random_uniform/max' type=Const>,
 <tf.Operation 'word_embedding/random_uniform/RandomUniform' type=RandomUniform>,
 <tf.Operation 'word_embedding/random_uniform/sub' type=Sub>,
 <tf.Operation 'word_embedding/random_uniform/mul' type=Mul>,
 <tf.Operation 'word_embedding/random_uniform' type=Add>,
 <tf.Operation 'word_embedding/W' type=VariableV2>,
 <tf.Operation 'word_embedding/W/Assign' type=Assign>,
 <tf.Operation 'word_embedding/W/read' type=Identity>,
 <tf.Operation 'word_embedding/embedding_lookup' t

In [9]:
sentence_support_df.head()

,pathwayA,pathwayB,crosstalk,pmid,sentenceFromPaper,label,tokenizedSentenceFromPaper,predictedLabel
0,Adipocytokine signaling pathway,ErbB signaling pathway,yes,23228483,"In the present study, we demonstrate that lept...",1,"in the present study, we demonstrate that path...",1
1,Adipocytokine signaling pathway,ErbB signaling pathway,yes,18945363,"In summary, our results suggest the existence ...",1,"in summary, our results suggest the existence ...",1
2,Adipocytokine signaling pathway,Estrogen signaling pathway,yes,20410173,These observations support the notion that the...,1,these observations support the notion that the...,1
3,Adipocytokine signaling pathway,Estrogen signaling pathway,yes,23357303,The crosstalk between leptin and estrogen resc...,1,the crosstalk between pathwayA and pathwayB re...,1
4,Adipocytokine signaling pathway,Estrogen signaling pathway,yes,22178935,The study supports the existence of a crosstal...,1,the study supports the existence of a crosstal...,1


In [11]:
from sklearn.model_selection import KFold

In [31]:
output = np.zeros(sentence_support_df.shape[0])

In [35]:
# Creating folds
kf = KFold(n_splits=4, random_state=5, shuffle=True)
for k, (train_index, test_index) in enumerate(kf.split(x, y)):
# for train_index, test_index in kf.split(x):
#     print("Fold: %s =>" % k,  "TRAIN:", train_index, "TEST:", test_index)
    x_train, x_dev = x[train_index], x[test_index]
    y_train, y_dev = y[train_index], y[test_index]
    
    train_word_distancesA = word_distancesA[train_index]
    train_word_distancesB = word_distancesB[train_index]
    
    test_word_distancesA = word_distancesA[test_index]
    test_word_distancesB = word_distancesB[test_index]
    
    train_pos_embedding = pos_embedding[train_index]
    test_pos_embedding = pos_embedding[test_index]
    
    print("Fold: %s =>" % k, "Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))
    
    def predict(x,y,word_distancesA, word_distancesB, pos_embedding):
        input_x = sess.graph.get_tensor_by_name('input_x:0')
        input_y = sess.graph.get_tensor_by_name('input_y:0')
        dropout_keep_prob = sess.graph.get_tensor_by_name('dropout_keep_prob:0')
        wA = sess.graph.get_tensor_by_name('word_distancesA:0')
        wB = sess.graph.get_tensor_by_name('word_distancesB:0')
        encoded_pos = sess.graph.get_tensor_by_name('encoded_pos:0')

        return sess.run(sess.graph.get_tensor_by_name('output/predictions:0'), feed_dict = {
                  input_x: x,
                  input_y: y,
                  dropout_keep_prob: 1.0,
                  wA: word_distancesA,
                  wB: word_distancesB,
                  encoded_pos: pos_embedding
                })
    output[test_index] = predict(x_dev, y_dev, test_word_distancesA, test_word_distancesB, test_pos_embedding)

Fold: 0 => Train/Dev split: 31795/10599
Fold: 1 => Train/Dev split: 31795/10599
Fold: 2 => Train/Dev split: 31796/10598
Fold: 3 => Train/Dev split: 31796/10598


In [37]:
sentence_support_df['predictedLabel'].describe()

count    42394.000000
mean         0.014578
std          0.119856
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: predictedLabel, dtype: float64

In [45]:
sentence_support_df['predictedLabel'] = pd.Series(output, dtype='int')

In [46]:
sentence_support_df['predictedLabel'].describe()

count    40626.000000
mean         0.026412
std          0.160358
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: predictedLabel, dtype: float64

In [47]:
sentence_support_df[sentence_support_df['predictedLabel'] == sentence_support_df['label']].shape

(39879, 8)

In [49]:
sentence_support_df['crosstalk'] = sentence_support_df['crosstalk'].replace(['unclear', 'no'], 0)
sentence_support_df['crosstalk'] = sentence_support_df['crosstalk'].replace(['yes'], 1)
sentence_support_df.head()

,pathwayA,pathwayB,crosstalk,pmid,sentenceFromPaper,label,tokenizedSentenceFromPaper,predictedLabel
0,Adipocytokine signaling pathway,ErbB signaling pathway,1,23228483,"In the present study, we demonstrate that lept...",1,"in the present study, we demonstrate that path...",1.0
1,Adipocytokine signaling pathway,ErbB signaling pathway,1,18945363,"In summary, our results suggest the existence ...",1,"in summary, our results suggest the existence ...",1.0
2,Adipocytokine signaling pathway,Estrogen signaling pathway,1,20410173,These observations support the notion that the...,1,these observations support the notion that the...,1.0
3,Adipocytokine signaling pathway,Estrogen signaling pathway,1,23357303,The crosstalk between leptin and estrogen resc...,1,the crosstalk between pathwayA and pathwayB re...,0.0
4,Adipocytokine signaling pathway,Estrogen signaling pathway,1,22178935,The study supports the existence of a crosstal...,1,the study supports the existence of a crosstal...,1.0


In [84]:
sentence_support_df.label.value_counts()

0    41776
1      618
Name: label, dtype: int64

In [58]:
sentence_support = sentence_support_df[['pathwayA', 'pathwayB','pmid', 'crosstalk', 'label', 'predictedLabel']]
documents = pd.DataFrame(sentence_support.groupby(['pathwayA', 'pathwayB','pmid']).sum()).reset_index()
documents['crosstalk'] = documents['crosstalk'].apply(lambda x: 1 if x>0 else 0)
documents['label'] = documents['label'].apply(lambda x: 1 if x>0 else 0)
documents['predictedLabel'] = documents['predictedLabel'].apply(lambda x: 1 if x>0 else 0)
documents.head()

,pathwayA,pathwayB,pmid,crosstalk,label,predictedLabel
0,Adherens junction,Adipocytokine signaling pathway,18537097,0,0,0
1,Adherens junction,Apoptosis,23375756,0,0,1
2,Adherens junction,Apoptosis,25488805,1,1,1
3,Adherens junction,Apoptosis,25546411,0,0,0
4,Adherens junction,Apoptosis,25725070,0,0,0


In [87]:
from sklearn.metrics import precision_score, classification_report, accuracy_score, roc_auc_score
precision_score(documents.label, documents.predictedLabel)  

0.58156911581569115

In [57]:
documents.predictedLabel.value_counts()

0.0    2353
1.0     591
2.0     164
3.0      40
4.0       6
5.0       2
Name: predictedLabel, dtype: int64

In [69]:
print(classification_report(documents.label, documents.predictedLabel, target_names=['non-sup', 'sup']))

             precision    recall  f1-score   support

    non-sup       0.96      0.88      0.92      2703
        sup       0.58      0.82      0.68       568

avg / total       0.89      0.87      0.87      3271



In [79]:
print(accuracy_score(documents.label, documents.predictedLabel))

0.866401712015


In [88]:
print(roc_auc_score(documents.label, documents.predictedLabel))

0.848938386144


#### Machine learning performance

accuracy_score 0.449939686369

recall_score 0.788321167883

precision_score 0.201869158879

f1_score 0.321428571429

In [80]:
pathway_pairs = pd.DataFrame(documents.groupby(['pathwayA', 'pathwayB']).sum()).reset_index()[['pathwayA', 'pathwayB', 'crosstalk', 'predictedLabel']]
pathway_pairs.drop_duplicates(inplace=True)
pathway_pairs.crosstalk = pathway_pairs.crosstalk.apply(lambda x: 1 if x>0 else 0)
pathway_pairs.predictedLabel = pathway_pairs.predictedLabel.apply(lambda x: 1 if x>0 else 0)
pathway_pairs.head()

,pathwayA,pathwayB,crosstalk,predictedLabel
0,Adherens junction,Adipocytokine signaling pathway,0,0
1,Adherens junction,Apoptosis,1,1
2,Adherens junction,ErbB signaling pathway,1,1
3,Adherens junction,Estrogen signaling pathway,1,1
4,Adherens junction,HIF-1 signaling pathway,0,0


In [81]:
print(classification_report(pathway_pairs.crosstalk, pathway_pairs.predictedLabel, target_names=['no crosstalk', 'crosstalk']))

              precision    recall  f1-score   support

no crosstalk       0.85      0.55      0.67       273
   crosstalk       0.72      0.92      0.81       345

 avg / total       0.78      0.76      0.75       618



In [85]:
print(accuracy_score(pathway_pairs.crosstalk, pathway_pairs.predictedLabel))

0.758899676375


In [89]:
print(roc_auc_score(pathway_pairs.crosstalk, pathway_pairs.predictedLabel))

0.737044115305


#### Machine learning performance

accuracy_score 0.548387096774

recall_score 0.5

precision_score 0.628571428571

f1_score 0.556962025316

In [ ]:
accuracy_score 0.548387096774
recall_score 0.5
precision_score 0.628571428571
f1_score 0.556962025316